In [1]:
#import go here
import numpy as np
from astropy.io import fits
from tqdm import tqdm
import pandas as pd
from scipy import interpolate

In [2]:
#files go here

#list of 1253's neighbors
list_of_neighbors = '/home/ade/Documents/Research/fittesting/neighborsfile/170723002601253snr.csv'
object_spectra = '/home/ade/Documents/Research/fittesting/testobject/1707230026012532.fits'
neighbors_spectra_path = '/home/ade/Documents/Research/fittesting/neighborsspectra/'


In [3]:
#function
def wav(c0, c1, x):
    return c0 + (c1*x)

In [4]:
#object wavelength and flux construction
spectra = fits.open(object_spectra, memmap = True)
f_obj = spectra[4].data #flux of the spectra
e_obj = spectra[1].data #error of flux
disp_start = spectra[4].header['CRVAL1'] - (spectra[4].header['CRPIX1'] - 1) * spectra[4].header['CDELT1'] #at which wavelength the dispersion starts
w_obj = wav(disp_start, spectra[4].header['CDELT1'], np.linspace(1, len(f_obj), len(f_obj)))

#now we have wavelength and flux of object

In [5]:
#load the list of neigbors file
df_neighbors = pd.read_csv(list_of_neighbors)

#selection based on parameter
#data of object
#teff 6363.256348, logg 3.489459, feh -0.020469

df_neighbors = df_neighbors[df_neighbors.teff <= 6363.256348 + 67]
df_neighbors = df_neighbors[df_neighbors.teff >= 6363.256348 - 67]
df_neighbors = df_neighbors[df_neighbors.logg <= 3.489459 + 0.12]
df_neighbors = df_neighbors[df_neighbors.logg >= 3.489459 - 0.12]
df_neighbors = df_neighbors[df_neighbors.fe_h <= -0.020469 + 0.034]
df_neighbors = df_neighbors[df_neighbors.fe_h >= -0.020469 - 0.034]
#df_neighbors = df_neighbors.reset_index(drop=True)


#selection base on SNR
df_neighbors = df_neighbors[df_neighbors.snr_c2_iraf >= 30]
df_neighbors = df_neighbors.reset_index(drop=True)

testobject = df_neighbors['sobject_id_1']

matrix_of_flux = []

min_wav = []
max_wav = []
#check the wavelength of each of neighbors spectra
for i in range(len(testobject)):
    #read the file that contains the neighbors
    spectra = fits.open(neighbors_spectra_path + str(testobject[i]) +'2.fits', memmap = True)
    
    #flux of the spectra
    f = spectra[4].data 
    
    #wavelength construction
    disp_start = spectra[4].header['CRVAL1'] - (spectra[4].header['CRPIX1'] - 1) * spectra[4].header['CDELT1'] #at which wavelength the dispersion starts
    w = wav(disp_start, spectra[4].header['CDELT1'], np.linspace(1, len(f), len(f)))
    
    #append min and max wavelength value
    min_wav.append(w[0])
    max_wav.append(w[-1])
    
    #for each of the neighbors spectra, interpolate the flux into object spectra wavelength
    #print wavelength of object
    #print('length of w_obj ' +  str(len(w_obj)))
    #print('length of w_neigh ' +  str(len(w)))
    #print('min of w_obj ' + str((w_obj[0])))
    #print('max of w_obj ' + str((w_obj[-1])))
    #print('min of w ' + str((w[0])))
    #print('max of w ' + str((w[-1])))
    #print("---")
    
    #interpolation = interpolate.interp1d(w, f)
    #f = interpolation(w_obj)
    
    #store the flux value from each spectra
    #matrix_of_flux.append(pd.Series(f, name=str(i)))
#matrix_of_flux = pd.concat(result, axis=1)
print("maximum of start wavelength " + str(max(min_wav)))
print("maximum of end wavelength " + str(min(max_wav)))

maximum of start wavelength 5650.287388803847
maximum of end wavelength 5871.183129442087


In [6]:
print('start of w_obj ' + str((w_obj[0])))
print('end of w_obj ' + str((w_obj[-1])))

start of w_obj 5648.0361118317705
end of w_obj 5872.002339735559


In [7]:
print(str(w_obj[min(range(len(w_obj)), key=lambda i: abs(w_obj[i]-max(min_wav)))]))
print(min(range(len(w_obj)), key=lambda i: abs(w_obj[i]-max(min_wav))))

nearest_index_max_start_wav = min(range(len(w_obj)), key=lambda i: abs(w_obj[i]-max(min_wav)))
if w_obj[nearest_index_max_start_wav] < max(min_wav):
    nearest_index_max_start_wav = nearest_index_max_start_wav + 1
print(w_obj[nearest_index_max_start_wav])
print(nearest_index_max_start_wav)

5650.278508741186
41
5650.3332013487325
42


In [8]:
print(str(w_obj[min(range(len(w_obj)), key=lambda i: abs(w_obj[i]-min(max_wav)))]))
print(min(range(len(w_obj)), key=lambda i: abs(w_obj[i]-min(max_wav))))

nearest_index_min_end_wav = min(range(len(w_obj)), key=lambda i: abs(w_obj[i]-min(max_wav)))
if w_obj[nearest_index_min_end_wav] > min(max_wav):
    nearest_index_min_end_wav = nearest_index_min_end_wav - 1
print(w_obj[nearest_index_min_end_wav])
print(nearest_index_min_end_wav)

5871.181950622358
4080
5871.181950622358
4080


In [9]:
w_obj = w_obj[nearest_index_max_start_wav:nearest_index_min_end_wav]
f_obj = f_obj[nearest_index_max_start_wav:nearest_index_min_end_wav]
e_obj = e_obj[nearest_index_max_start_wav:nearest_index_min_end_wav]

e_obj = e_obj.tolist()

matrix_of_flux = []
#check the wavelength of each of neighbors spectra
for i in tqdm(range(len(testobject))):
    #read the file that contains the neighbors
    spectra = fits.open(neighbors_spectra_path + str(testobject[i]) +'2.fits', memmap = True)
    
    #flux of the spectra
    f = spectra[4].data 
    
    #wavelength construction
    disp_start = spectra[4].header['CRVAL1'] - (spectra[4].header['CRPIX1'] - 1) * spectra[4].header['CDELT1'] #at which wavelength the dispersion starts
    w = wav(disp_start, spectra[4].header['CDELT1'], np.linspace(1, len(f), len(f)))
    
    #print(w[0])
    #nterpolate to selected range object spectra
    interpolation = interpolate.interp1d(w, f)
    f = interpolation(w_obj)
    
    matrix_of_flux.append(pd.Series(f, name=str(i)))


100%|██████████| 10/10 [00:00<00:00, 116.82it/s]


In [10]:
result = pd.concat(matrix_of_flux, axis=1)

In [11]:
result = result.assign(w_obj = w_obj, f_obj = f_obj, e_obj = e_obj, f_mean = result.mean(1), f_stddev = result.std(1))

In [ ]:
#all_neighbors = pd.read_csv('model_spectra.csv')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(w_obj, f_obj, 'k')
plt.plot(w_obj, result.f_mean)
#plt.plot(w_obj, all_neighbors.f_mean +0.2, 'r')
plt.ylim(0, 1.5)
plt.xlim(5760, 5800)

In [12]:
result.to_csv('model_spectra_30_datarelease.csv', columns = ["w_obj", "f_obj", "e_obj","f_mean", "f_stddev"])

In [ ]:
plt.plot(w_obj, f_obj/result.f_mean, 'k')
plt.plot(w_obj, f_obj + 0.4, 'b')
plt.ylim(0, 1.5)
plt.xlim(5750, 5810)

In [ ]:
import matplotlib.pyplot as plt


plt.plot(w_obj, f_obj, 'k')
plt.plot(w_obj, result.f_mean)
#plt.plot(w_obj, all_neighbors.f_mean +0.2, 'r')
plt.ylim(0, 1.5)
plt.xlim(5750, 5810)